In [1]:
# TODO: Fix use of arrays in the dictionaries to just values
import re
import ast
import time
import requests
import pandas as pd
import numpy as np
import os.path
import urllib.request
import concurrent.futures
from os import listdir
from stem import Signal
from requests import get
from bs4 import BeautifulSoup
from datetime import datetime
from os.path import isfile, join
from stem.control import Controller

pickles_save_dir = "./pickles"

In [2]:
def get_tor_session():
    session = requests.session()
    # Tor uses the 9050 port as the default socks port
    session.proxies = {'http':  'socks5://127.0.0.1:9050',
                       'https': 'socks5://127.0.0.1:9050'}
    return session

def renew_connection():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="password")
        controller.signal(Signal.NEWNYM)

In [4]:
if os.path.exists('./pickles/filtered_movies.pkl'):
    movies = pd.read_pickle("./pickles/filtered_movies.pkl")
else:
    title_basics = pd.read_csv("data/title.basics.tsv.gz", sep='\t')
    movies = title_basics[title_basics.titleType == 'movie']
    if not os.path.exists(pickles_save_dir):
        os.makedirs(pickles_save_dir)
    movies.to_pickle("{0}/filtered_movies.pkl".format(pickles_save_dir))
movies.head()

/home/truchiranga/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
14,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
15,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
238,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
552,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama"
825,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N


In [5]:
len(movies)

893905

In [13]:
session = get_tor_session()
url = 'https://www.imdb.com/title/tt0120338/fullcredits'
data = get_data(session, url)
print(data)
print(len(data['cast']))

{'tconst': 'tt0120338', 'cast': ['nm0000138', 'nm0000701', 'nm0000708', 'nm0000870', 'nm0004920', 'nm0001784', 'nm0000200', 'nm0384060', 'nm0001831', 'nm0001255', 'nm0404993', 'nm0000751', 'nm0008656', 'nm0143167', 'nm0755972', 'nm0001572', 'nm0058029', 'nm0829380', 'nm0344435', 'nm0680526', 'nm0152464', 'nm0334262', 'nm0109229', 'nm0231871', 'nm0000967', 'nm0154195', 'nm0288882', 'nm0258117', 'nm0107938', 'nm0001280', 'nm0653829', 'nm0449599', 'nm0301401', 'nm0419089', 'nm0044011', 'nm0741645', 'nm0263323', 'nm0909574', 'nm0853092', 'nm0654302', 'nm0186426', 'nm0281992', 'nm0027424', 'nm0247542', 'nm0446287', 'nm0177427', 'nm0876720', 'nm0484140', 'nm0712341', 'nm0658817', 'nm0859809', 'nm0484837', 'nm0905290', 'nm0905289', 'nm0874171', 'nm0906975', 'nm0286157', 'nm0494749', 'nm0039263', 'nm0626119', 'nm0175267', 'nm0188836', 'nm0934350', 'nm0481918', 'nm0289211', 'nm0575164', 'nm0654503', 'nm0135633', 'nm0006976', 'nm0378609', 'nm0416474', 'nm0126137', 'nm0656297', 'nm0308127', 'nm03

In [14]:
def get_data(session, url):
    info=[]
    
    
    r = session.get(url)
    r.raise_for_status()
    
    page_body = r.text
    soup = BeautifulSoup(page_body, 'html.parser')
    
    #tconst
    tconst=url.rsplit('/', 2)[-2]
    
    scraped_data = {
        "tconst": tconst,
        "cast": []
    }
    
    table = soup.find('table', class_='cast_list')
    if table is not None:
        atags = table.find_all('a')
        if atags is not None:
            for atag in atags:
                nconst_res = re.search('nm\d{7}', atag.get('href'))
                if nconst_res is not None:
                    scraped_data['cast'].append(nconst_res.group(0))
            
    # Drop duplicates
    scraped_data['cast'] = list(dict.fromkeys(scraped_data['cast']))
    scraped_data['count'] = len(scraped_data['cast'])
    return scraped_data

In [11]:
# Set the start index and end index. Ip is renewed after each chunk of urls of 'step_size'. 
# After every chunk, result is written to a pickle. So if you need to stop the execution in the middle, note the
# index range of the last successfully written pickle file (from the printed logs) and use the remaining range
# for start and end index to resume from where you stopped.
postfix = 'tor'
s_idx = 0
e_idx = len(movies)
step_size = 4000

base_url = 'https://www.imdb.com/title/{0}/fullcredits'
urls=[]
for index, row in movies[s_idx:e_idx].iterrows():
    urls.append(base_url.format(row['tconst']))

used_ips = []
failed_urls = []

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    session = get_tor_session()
    chunk_indeces = np.arange(0, len(urls), step_size)
    
    for idx, start in enumerate(chunk_indeces, start=1):
        start_t = time.time()
        finals = []
        renew_connection()
        session = get_tor_session()
        new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
        while new_ip in used_ips:
            print("Renewed IP {0} already used. Waiting 5s to renew...".format(new_ip))
            time.sleep(5)
            renew_connection()
            session = get_tor_session()
            new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
        used_ips.append(new_ip)
        
        url_chunk = urls[start:start + step_size] + failed_urls
        failed_urls = []
        
        future_to_url = {executor.submit(get_data, session, url): url for url in url_chunk}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                data = future.result()
                finals.append(data)
            except Exception as exc:
                failed_urls.append(url.strip("'"))
                print('%s generated an exception: %s. Added to retry list.' % (url, exc))
        
        concurrent.futures.wait(
            list(future_to_url.keys()), 
            timeout=None, 
            return_when=concurrent.futures.ALL_COMPLETED)
        end_t = time.time()
        print("chunk {0} of {1} completed with IP {2}. Took {3} seconds."\
              .format(idx, len(chunk_indeces), new_ip, end_t - start_t))   
        cast_df = pd.DataFrame(data=finals)
        cast_pickles_savedir = "{0}/casttor".format(pickles_save_dir)
        if not os.path.exists(cast_pickles_savedir):
            os.makedirs(cast_pickles_savedir)
        pickle_path = "{4}/{0}_{1}-{2}_{3}.pkl".format(idx, start, start + step_size, postfix, cast_pickles_savedir)
        cast_df.to_pickle(pickle_path)
        print("Wrote pickle {0} with {1} rows".format(pickle_path, len(cast_df)))
        
        
if len(failed_urls) > 0:
    print('\n\n\n\n!!!**********************************!!!')
    print('Failed to retrieve the following URLs.\n')
    print(failed_urls)
    print('\n!!!**********************************!!!')
    print('Note: Run the following cell to retry the failed urls')
else:
    print("\n\n\n\n!!!**********************************!!!\n")
    print("Scraping succeeded!")
    print('\n!!!**********************************!!!')
    print('''Note: Next, run the script that combine all pickles''')

chunk 1 of 224 completed with IP 46.165.245.154. Took 322.17992091178894 seconds.
Wrote pickle ./pickles/casttor/1_0-4000_tor.pkl with 4000 rows
chunk 2 of 224 completed with IP 130.149.80.199. Took 271.98430728912354 seconds.
Wrote pickle ./pickles/casttor/2_4000-8000_tor.pkl with 4000 rows
chunk 3 of 224 completed with IP 109.69.67.17. Took 673.3516044616699 seconds.
Wrote pickle ./pickles/casttor/3_8000-12000_tor.pkl with 4000 rows
chunk 4 of 224 completed with IP 104.244.76.56. Took 275.9324903488159 seconds.
Wrote pickle ./pickles/casttor/4_12000-16000_tor.pkl with 4000 rows
chunk 5 of 224 completed with IP 162.247.72.199. Took 289.9110713005066 seconds.
Wrote pickle ./pickles/casttor/5_16000-20000_tor.pkl with 4000 rows
https://www.imdb.com/title/tt0019469/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0019470/fullcredits generated an exception: ('C

https://www.imdb.com/title/tt0019438/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0019453/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0019471/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0019472/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0019471/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0019474/fullcredits generated a

chunk 14 of 224 completed with IP 51.38.99.96. Took 370.65647196769714 seconds.
Wrote pickle ./pickles/casttor/14_52000-56000_tor.pkl with 4000 rows
chunk 15 of 224 completed with IP 185.220.102.7. Took 355.7592136859894 seconds.
Wrote pickle ./pickles/casttor/15_56000-60000_tor.pkl with 4000 rows
chunk 16 of 224 completed with IP 162.247.74.204. Took 400.10620617866516 seconds.
Wrote pickle ./pickles/casttor/16_60000-64000_tor.pkl with 4000 rows
chunk 17 of 224 completed with IP 185.220.101.75. Took 372.41557240486145 seconds.
Wrote pickle ./pickles/casttor/17_64000-68000_tor.pkl with 4000 rows
chunk 18 of 224 completed with IP 89.31.57.5. Took 379.537700176239 seconds.
Wrote pickle ./pickles/casttor/18_68000-72000_tor.pkl with 4000 rows
chunk 19 of 224 completed with IP 185.220.100.255. Took 357.32702231407166 seconds.
Wrote pickle ./pickles/casttor/19_72000-76000_tor.pkl with 4000 rows
chunk 20 of 224 completed with IP 185.220.100.253. Took 402.90447092056274 seconds.
Wrote pickle .

https://www.imdb.com/title/tt0169638/fullcredits generated an exception: SOCKSHTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt0169638/fullcredits (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))). Added to retry list.
https://www.imdb.com/title/tt0169635/fullcredits generated an exception: SOCKSHTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /title/tt0169635/fullcredits (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')"))). Added to retry list.
https://www.imdb.com/title/tt0169640/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0174642/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
chunk 45 of 224 completed with I

https://www.imdb.com/title/tt0257358/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0257356/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0257376/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0257285/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0257344/fullcredits generated an exception: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')). Added to retry list.
https://www.imdb.com/title/tt0257362/fullcredits generated a

...

503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9573812/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9578212/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9573652/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9579488/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9579386/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9579404/fullcredits503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9573612/fullcredits503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9579386/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9579634/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9573684/fullcredits
503 Server Error: Service Unavailable for 

503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9581076/fullcredits503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9581826/fullcredits

503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9581126/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9581736/fullcredits503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9576082/fullcredits

503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9581780/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9581128/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9576076/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9580982/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9575726/fullcredits
503 Server Error: Service Unavailable fo

503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9584718/fullcredits503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9584718/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9585496/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9585142/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9585788/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9585186/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9585696/fullcredits

503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9585692/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9585676/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9585152/fullcredits
503 Server Error: Service Unavailable fo

503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9586942/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9587018/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9588466/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9588466/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9587040/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9587388/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9588480/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9588298/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9587244/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9588506/fullcredits
503 Server Error: Service Unavailable fo

503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9590916/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9590950/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9591696/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9591696/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9591766/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9590954/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9591004/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9591692/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9591532/fullcredits
503 Server Error: Service Unavailable for url: https://www.imdb.com/title/tt9590826/fullcredits503 Server Error: Service Unavailable for

In [ ]:
'''
Run the following script only if there were any failed URLS at the end of the previous script execution.
Run this several times if you still see failures until there are none. But if some dont succeed even 
after multiple attempts, its probably a problem on imdb side and we can just ignore them.
'''
# urls = failed_urls

# if len(urls) > 0:
#     retry_failed_urls = []

#     with concurrent.futures.ThreadPoolExecutor(max_workers=150) as executor:
#         session = get_tor_session(None)
#         chunk_indeces = np.arange(0, len(urls), step_size)

#         for idx, start in enumerate(chunk_indeces, start=1):
#             start_t = time.time()
#             finals = []
#             renew_connection()
#             session = get_tor_session(session)
#             new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
#             while new_ip in used_ips:
#                 print("Renewed IP {0} already used. Waiting 5s to renew...".format(new_ip))
#                 time.sleep(5)
#                 renew_connection()
#                 session = get_tor_session(session)
#                 new_ip = ast.literal_eval(session.get("http://httpbin.org/ip").text)["origin"].split(",")[0]
#             used_ips.append(new_ip)

#             url_chunk = urls[start:start + step_size] + retry_failed_urls
#             retry_failed_urls = []

#             future_to_url = {executor.submit(get_data, session, url): url for url in url_chunk}
#             for future in concurrent.futures.as_completed(future_to_url):
#                 url = future_to_url[future]
#                 try:
#                     data = future.result()
#                     finals.append(data)
#                 except Exception as exc:
#                     retry_failed_urls.append(url.strip("'"))
#                     print('%s generated an exception: %s. Added to retry list.' % (url, exc))

#             concurrent.futures.wait(list(future_to_url.keys()), timeout=None, return_when=concurrent.futures.ALL_COMPLETED)
#             end_t = time.time()
#             print("chunk {0} of {1} completed with IP {2}. Took {3} seconds.".format(idx, len(chunk_indeces), new_ip, end_t - start_t))   
#             cast_df = pd.DataFrame(data=finals)
#             if len(movies_df) > 0:
#                 cast_pickles_savedir = "{0}/casttor".format(pickles_save_dir)
#                 if not os.path.exists(cast_pickles_savedir):
#                     os.makedirs(cast_pickles_savedir)
#                 pickle_path = "{4}/{0}_{1}-{2}_{3}_retry.pkl".format(idx, start, start + step_size, postfix, cast_pickles_savedir)
#                 cast_df.to_pickle(pickle_path)
#                 print("Wrote pickle {0} with {1} rows".format(pickle_path, len(cast_df)))

#         if len(retry_failed_urls) > 0:
#             print('\n\n\n\n!!!**********************************!!!')
#             print('Retry failed with the following URLs.\n')
#             print(retry_failed_urls)
#             print('\n!!!**********************************!!!')
#             print('Note: Run this script several times to see if all succeed.',
#             'If some dont, it probably means they wont ever. so just give them up.')
#         else:
#             print("\n\n\n\n!!!**********************************!!!\n")
#             print("Retry succeeded!")
#             print('\n!!!**********************************!!!')
#             print('''Note: Run the following script to combine all pickles''')

In [10]:
# Combine the created pickles into one Dataframe

dir_path = "{0}/casttor".format(pickles_save_dir)
# Ignore any sytem files starting with . or folders if any
pickles = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) and not f.startswith('.')]
dfs = list(map(lambda x: pd.read_pickle("{0}/{1}".format(dir_path,x)), pickles))
combined_df = pd.concat(dfs)
combined_df.reset_index(drop=True, inplace=True)
combined_df.to_pickle("./pickles/complete_cast{0}.pkl".format(e_idx))
len(combined_df)
with pd.option_context('display.max_rows', 50000, 'display.max_columns', 20):
    display(combined_df)


,tconst,cast,count
0,tt0000502,"[nm0215752, nm0252720]",2
1,tt0000009,"[nm0063086, nm0183823, nm1309758]",3
2,tt0000009,"[nm0063086, nm0183823, nm1309758]",3
3,tt0000574,"[nm0846887, nm0846894, nm3002376, nm0170118, n...",15
4,tt0000574,"[nm0846887, nm0846894, nm3002376, nm0170118, n...",15
5,tt0000147,"[nm0179163, nm0280615, nm4082222, nm4081458, n...",5
6,tt0000502,"[nm0215752, nm0252720]",2
7,tt0000679,"[nm0000875, nm0122665, nm0933446, nm2924919, n...",34
8,tt0000675,[],0
9,tt0000679,"[nm0000875, nm0122665, nm0933446, nm2924919, n...",34
